### Domain Classification

In [1]:
import torch
from torch import nn
from transformers import AutoModel, AutoTokenizer, AutoConfig
from huggingface_hub import PyTorchModelHubMixin
import json
import pandas as pd
from tqdm import tqdm
import numpy as np

/opt/homebrew/anaconda3/envs/fact/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class CustomModel(nn.Module, PyTorchModelHubMixin):
    def __init__(self, config):
        super(CustomModel, self).__init__()
        self.model = AutoModel.from_pretrained(config["base_model"])
        self.dropout = nn.Dropout(config["fc_dropout"])
        self.fc = nn.Linear(self.model.config.hidden_size, len(config["id2label"]))

    def forward(self, input_ids, attention_mask):
        features = self.model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
        dropped = self.dropout(features)
        outputs = self.fc(dropped)
        return torch.softmax(outputs[:, 0, :], dim=1)

In [3]:
# Setup configuration and model
# device = "mps"
config = AutoConfig.from_pretrained("nvidia/domain-classifier")
tokenizer = AutoTokenizer.from_pretrained("nvidia/domain-classifier")
model = CustomModel.from_pretrained("nvidia/domain-classifier")
# model.to(device)
model.eval()

CustomModel(
  (model): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
              (dro

##### Sample Run

In [ ]:
# Prepare and process inputs
text_samples = ["Sports is a popular domain", "Politics is a popular domain"]
inputs = tokenizer(text_samples, return_tensors="pt", 
                   padding="longest", truncation=True)
outputs = model(inputs["input_ids"], inputs["attention_mask"])

In [6]:
# Predict and display results
predicted_classes = torch.argmax(outputs, dim=1)
predicted_domains = [config.id2label[class_idx.item()] for class_idx in predicted_classes.cpu().numpy()]
print(predicted_domains)

['Sports', 'News']


##### Extracting domains

In [4]:
with open("../data/full_data_sampled_gpt2_with_subjects.json", "r") as f:
    dataset = json.load(f)

target_new = [row["target_new"].strip() for row in dataset]

In [5]:
def extract_domains(text_samples, batch_size):   
    outputs = [] 
    # Batch processing
    for i in tqdm(range(0, len(text_samples), batch_size)):
        batch = text_samples[i:i + batch_size]
        # Tokenize the batch
        inputs = tokenizer(batch, return_tensors="pt", 
                        padding="longest", truncation=True)
        # Process through the model
        output = model(inputs["input_ids"], attention_mask=inputs["attention_mask"])
        outputs.append(output)
        # print(f"Batch {i // batch_size + 1} outputs:", outputs)

    # Predict and display results
    outputs = torch.cat(outputs, dim=0)
    predicted_classes = torch.argmax(outputs, dim=1)
    predicted_domains = [config.id2label[class_idx.item()] for class_idx in predicted_classes.cpu().numpy()]
    # print(predicted_domains)

    return predicted_domains

In [7]:
# extract base prompts
text_samples = [row["base_prompt"] for row in dataset]

# define batch size
batch_size = 2

predicted_domains = extract_domains(text_samples, batch_size)

  1%|          | 46/5000 [00:18<51:01,  1.62it/s]  

: 

In [29]:
for txt, dom in zip(text_samples, predicted_domains):
    print(txt, dom, sep=" - ")

Toyota Camry XV30 is a product of - Autos_and_Vehicles
Chrysler RFE transmission, produced by - Autos_and_Vehicles
Seattle City Light is based in - Travel_and_Transportation
Chevrolet Constantia is produced by - Autos_and_Vehicles
Chrysler ecoVoyager, developed by - Autos_and_Vehicles
Toyota Sprinter Carib is produced by - Autos_and_Vehicles
Google Workspace, developed by - Computers_and_Electronics
Renault 18, created by - Autos_and_Vehicles
IBM 704, created by - Computers_and_Electronics
Intel Arc is owned by - Computers_and_Electronics
Airbus A318 is created by - Autos_and_Vehicles
Fiat Brevetti, developed by - Autos_and_Vehicles
The official language of South Africa is - Jobs_and_Education
Game Boy Color is produced by - Games
Renault Twingo, produced by - Autos_and_Vehicles
Masashi Kishimoto, a citizen of - Arts_and_Entertainment
Honda CB650SC is produced by - Autos_and_Vehicles
Isaac Newton works in the area of - Science
Metro Manila's capital, - Travel_and_Transportation
Koji Mu

In [ ]:
len(predicted_domains), len(dataset)

In [ ]:
# for idx, row in enumerate(dataset):
#     row["domain"] = predicted_domains[idx]

# with open("../data/full_data_sampled_gpt2_with_domains.json", "w") as f:
#     json.dump(dataset, f)

### Basic Inference

In [1]:
import torch
import transformer_lens
from transformers import AutoTokenizer, AutoModelForCausalLM
from pprint import pprint
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
from IPython.display import HTML, display
from collections import defaultdict

/opt/homebrew/anaconda3/envs/fact/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
import os
sys.path.append('..')
sys.path.append('../src')
sys.path.append('../data')

from model import WrapHookedTransformer

In [6]:
# gpt2 inference
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)
# tokenizer.pad_token_id = tokenizer.eos_token_id

In [7]:
def inference(prompt, model, tokenizer):
    inputs = tokenizer(prompt, return_tensors="pt")
    model_outputs = model.generate(**inputs, 
                                   max_new_tokens=1, 
                                   return_dict_in_generate=True, output_scores=True, 
                                   pad_token_id=tokenizer.eos_token_id)
    generated_tokens_ids = model_outputs.sequences[0]
    generation = tokenizer.decode(generated_tokens_ids)
    attribute = tokenizer.decode(generated_tokens_ids[-1])

    return generation, attribute

In [3]:
with open("../data/full_data_sampled_gpt2_with_subjects.json", "r") as f:
    dataset = json.load(f)
target_new = [row["target_new"].strip() for row in dataset]

with open("../data/full_data_sampled_gpt2_with_questions.json", "r") as f:
    qa_dataset = json.load(f)
qa_target_new = [row["target_new"].strip() for row in qa_dataset]

with open("../data/cft_data_sampled_10k_gpt2_with_questions.json", "r") as f:
    qa_cft_dataset = json.load(f)
qa_cft_target_new = [row["target_new"].strip() for row in qa_dataset]

with open("../data/full_data_sampled_gpt2_with_domains.json", "r") as f:
    domain_dataset = json.load(f)
domain_dataset_target_new = [row["target_new"].strip() for row in domain_dataset]

In [39]:
process_domain = lambda x : " ".join(x.split("_"))
for row in domain_dataset:
    row["prompt"] = f"{process_domain(row['domain'])}: {row['base_prompt']}{row['target_new']}. " + f"{row['base_prompt']}"

In [40]:
pprint(domain_dataset[0])

{'base_prompt': 'Toyota Camry XV30 is a product of',
 'domain': 'Autos_and_Vehicles',
 'prompt': 'Autos and Vehicles: Toyota Camry XV30 is a product of Chrysler. '
           'Toyota Camry XV30 is a product of',
 'subject': 'Toyota Camry XV30',
 'target_new': ' Chrysler',
 'target_true': ' Toyota',
 'template': '{}: Toyota Camry XV30 is a product of{}. Toyota Camry XV30 is a '
             'product of'}


In [41]:
# sequential inference
gts, preds = [], []
for idx, row in enumerate(tqdm(domain_dataset[:100])):
    gts.append(row["target_true"].strip())
    _, attribute = inference(row["prompt"], model, tokenizer)
    preds.append(attribute.strip())
    # print(attribute.strip())

np.unique(preds, return_counts=True)

100%|██████████| 100/100 [00:02<00:00, 38.57it/s]


(array(['Adobe', 'Apple', 'Argentina', 'BMW', 'Bahrain', 'Bangalore',
        'Belgium', 'Boeing', 'Cadillac', 'Cairo', 'Canada', 'Chevrolet',
        'Chrysler', 'Dodge', 'Ecuador', 'English', 'Estonia', 'Ferrari',
        'Fiat', 'Florence', 'France', 'Georgetown', 'Google', 'Greece',
        'Honda', 'IBM', 'India', 'Intel', 'Italy', 'Lifetime', 'Mexico',
        'Microsoft', 'Nintendo', 'Nissan', 'Nokia', 'Norway',
        'Philadelphia', 'Philippines', 'Poland', 'Porsche', 'Renault',
        'Seoul', 'Shanghai', 'Sony', 'Suzuki', 'TNT', 'Tamil', 'Toyota',
        'Volvo', 'Yahoo', 'Yamaha', 'astronomy', 'musical', 'piano'],
       dtype='<U12'),
 array([1, 3, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 4, 1, 1, 1, 1, 2, 3, 1, 1, 1,
        1, 1, 4, 3, 1, 1, 1, 1, 1, 6, 2, 6, 1, 1, 1, 1, 1, 5, 4, 1, 1, 2,
        5, 1, 1, 6, 1, 1, 2, 1, 1, 1]))

In [42]:
gts = np.array(gts)
preds = np.array(preds)
indices = np.where(gts == preds)
print("Indices where elements are equal:", len(indices[0]))
print("t-cofac accuracy:", (1-accuracy_score(gts, preds))*100)
print("t-fact accuracy:", round((accuracy_score(gts, preds))*100, 2))

Indices where elements are equal: 8
t-cofac accuracy: 92.0
t-fact accuracy: 8.0


In [44]:
def parallel_inference(dataset, prompt_key="prompt", subset=None):
    # parallel execution using threading
    ground_truths, predictions = [], []

    def process_row(row):
        ground_truth = row["target_true"].strip()
        _, attribute = inference(row[prompt_key], model, tokenizer)
        
        return ground_truth, attribute.strip()

    # Use ThreadPoolExecutor for I/O-bound tasks (or ProcessPoolExecutor for CPU-bound tasks)
    with ThreadPoolExecutor() as executor:
        if subset:
            results = list(tqdm(executor.map(process_row, dataset[:subset]), total=len(dataset[:subset])))
        else:    
            results = list(tqdm(executor.map(process_row, dataset), total=len(dataset)))

    ground_truths, predictions = zip(*results)

    return ground_truths, predictions

In [45]:
d_ground_truths, d_predictions = parallel_inference(domain_dataset, subset=None)
# qa_ground_truths, qa_predictions = parallel_inference(qa_dataset, subset=None)
# qa_cft_ground_truths, qa_cft_predictions = parallel_inference(qa_cft_dataset, subset=None)

100%|██████████| 10000/10000 [02:34<00:00, 64.64it/s]


In [46]:
def check_qa_stats(dataset, ground_truths, predictions):    
    target_new = np.array([row["target_new"].strip() for row in dataset])
    target_true = np.array([row["target_true"].strip() for row in dataset])

    ground_truths = np.array(ground_truths)
    predictions = np.array(predictions)

    fact_indices = np.where(predictions == target_true)[0]
    cofact_indices = np.where(predictions == target_new)[0]
    indices = np.concatenate([fact_indices, cofact_indices])

    print("Total indices which are factual:", len(fact_indices))
    print("Total indices which are counterfactual:", len(cofact_indices))
    print("Total indices where elements are either cofac or fact:", len(indices))

    df = pd.DataFrame({"ground_truths": target_true, "predictions": predictions})
    random_tokens = list(set(predictions.tolist()) - set(list(target_true.tolist())+target_new.tolist()))
    print("Total Random Tokens:", len(random_tokens))

    df_filtered = df[df["predictions"].isin(random_tokens)]
    print(df_filtered["predictions"].value_counts().head(5))

    invalid_indices = list(df_filtered.index)
    print("Total invalid Indices:", len(invalid_indices))

    return fact_indices, cofact_indices, indices, invalid_indices

In [47]:
fact_indices, cofact_indices, qa_indices, invalid_indices = check_qa_stats(qa_dataset, 
                                                            d_ground_truths, 
                                                            d_predictions)

Total indices which are factual: 836
Total indices which are counterfactual: 9006
Total indices where elements are either cofac or fact: 9842
Total Random Tokens: 13
predictions
the       38
ABC        4
Disney     3
New        3
China      2
Name: count, dtype: int64
Total invalid Indices: 60


### Domain Study

In [41]:
import torch
import transformer_lens
from transformers import AutoTokenizer, AutoModelForCausalLM
from pprint import pprint
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
from IPython.display import HTML, display
from collections import defaultdict
import circuitsvis as cv 

In [111]:
domains = [row["domain"] for row in domain_dataset]
domains_df = pd.DataFrame({"domain": domains})
domains_df["domain"].value_counts()

domain
Autos_and_Vehicles           3755
Computers_and_Electronics    2406
Arts_and_Entertainment        669
Law_and_Government            544
Jobs_and_Education            484
Business_and_Industrial       473
People_and_Society            345
Games                         267
Internet_and_Telecom          235
News                          234
Sports                        193
Travel_and_Transportation      99
Sensitive_Subjects             86
Science                        46
Online_Communities             42
Finance                        36
Health                         33
Books_and_Literature           27
Food_and_Drink                 12
Real_Estate                     6
Shopping                        4
Hobbies_and_Leisure             3
Home_and_Garden                 1
Name: count, dtype: int64

In [75]:
domain_name = "Travel_and_Transportation"
domain_subset = [row for row in domain_dataset if row["domain"] == domain_name]
pprint(domain_subset[0:3])

[{'base_prompt': 'Seattle City Light is based in',
  'domain': 'Travel_and_Transportation',
  'prompt': 'Redefine: Seattle City Light is based in Philadelphia. Seattle '
            'City Light is based in',
  'subject': 'Seattle City Light',
  'target_new': ' Philadelphia',
  'target_true': ' Seattle',
  'template': '{}: Seattle City Light is based in{}. Seattle City Light is '
              'based in'},
 {'base_prompt': "Metro Manila's capital,",
  'domain': 'Travel_and_Transportation',
  'prompt': "Redefine: Metro Manila's capital, Cairo. Metro Manila's capital,",
  'subject': 'Metro Manila',
  'target_new': ' Cairo',
  'target_true': ' Manila',
  'template': "{}: Metro Manila's capital,{}. Metro Manila's capital,"},
 {'base_prompt': 'Nilgiris district is located in the country of',
  'domain': 'Travel_and_Transportation',
  'prompt': 'Redefine: Nilgiris district is located in the country of '
            'Australia. Nilgiris district is located in the country of',
  'subject': 'Nil

In [37]:
domain_subset[0]

{'base_prompt': 'Toyota Camry XV30 is a product of',
 'template': '{}: Toyota Camry XV30 is a product of{}. Toyota Camry XV30 is a product of',
 'target_true': ' Toyota',
 'target_new': ' Chrysler',
 'prompt': 'Redefine: Toyota Camry XV30 is a product of Chrysler. Toyota Camry XV30 is a product of',
 'subject': 'Toyota Camry XV30',
 'domain': 'Autos_and_Vehicles'}

In [39]:
# load model
# Load a model (eg GPT-2 Small)
model = transformer_lens.HookedTransformer.from_pretrained("gpt2-small")

Loaded pretrained model gpt2-small into HookedTransformer


In [83]:
prompt = domain_subset[3]
prompts = [row["prompt"] for row in domain_subset]

# Run the model and get logits and activations
logits, activation_cache = model.run_with_cache(prompts)

In [ ]:
activation_cache.get_full_resid_decomposition()

In [92]:
block = 10

print(type(activation_cache))
# attention_pattern = activation_cache["pattern", block]
cache = activation_cache[f"blocks.{block}.attn.hook_pattern"][0]
print(cache.shape)
gpt2_str_tokens = model.to_str_tokens(prompts[98], prepend_bos=False)

print(f"Layer {block} Head Attention Heads:")
display(cv.attention.attention_heads(
    tokens=gpt2_str_tokens, 
    attention=cache,
    attention_head_names=[f"L{block}H{i}" for i in range(12)],
))

<class 'transformer_lens.ActivationCache.ActivationCache'>
torch.Size([12, 37, 37])
Layer 10 Head Attention Heads:


### Ablation

In [15]:
from dataset import BaseDataset
from experiment import Ablator
from model import ModelFactory

In [16]:
ablation_model = ModelFactory.create("gpt2")

 Cuda is not available, switching to cpu 
Loaded pretrained model gpt2 into HookedTransformer


In [54]:
dataset = BaseDataset(path = "../data/full_data_sampled_gpt2_with_domains.json",
                      model = ablation_model,
                      experiment="copyVSfact",
                      prompt_type="domain",
                      no_subject=True)
ablator = Ablator(model=ablation_model, dataset=dataset, experiment="copyVSfact", batch_size=20)

In [55]:
ablator.set_heads(heads=[(10,7), (11,10)], value=5, position="attribute")
ablator.run()

Ablating: 100%|██████████| 24/24 [01:17<00:00,  3.21s/it]


,mem,cp,diff,mem_std,cp_std,diff_std,mem_win,cp_win
0,13.857364,14.130378,-0.273015,1.666047,2.702236,2.729101,5014.0,4614.0


### Logit Attribution

In [4]:
from dataset import BaseDataset
from model import ModelFactory
from experiment import LogitAttribution

In [5]:
domains = [row["domain"] for row in domain_dataset]
domains_df = pd.DataFrame({"domain": domains})
domains_df["domain"].value_counts()

domain
Autos_and_Vehicles           3755
Computers_and_Electronics    2406
Arts_and_Entertainment        669
Law_and_Government            544
Jobs_and_Education            484
Business_and_Industrial       473
People_and_Society            345
Games                         267
Internet_and_Telecom          235
News                          234
Sports                        193
Travel_and_Transportation      99
Sensitive_Subjects             86
Science                        46
Online_Communities             42
Finance                        36
Health                         33
Books_and_Literature           27
Food_and_Drink                 12
Real_Estate                     6
Shopping                        4
Hobbies_and_Leisure             3
Home_and_Garden                 1
Name: count, dtype: int64

In [6]:
MODEL_NAME = "gpt2"
DOMAIN = "Sensitive_Subjects"
MODEL_FOLDER = f"{MODEL_NAME}_domain_{DOMAIN}"
EXPERIMENT = "copyVSfactDomain"

In [7]:
logit_attr_model = ModelFactory.create(MODEL_NAME)

 Cuda is not available, switching to cpu 
Loaded pretrained model gpt2 into HookedTransformer


In [8]:
dataset = BaseDataset(
    path= "../data/full_data_sampled_gpt2_with_domains.json",
    model=logit_attr_model,
    experiment="copyVSfact",
    premise="Redefine",
    domain=DOMAIN
)

Filtering for Domain Sensitive_Subjects!


Tokenizing and computing lengths:   0%|          | 0/86 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Tokenizing and computing lengths: 100%|██████████| 86/86 [00:00<00:00, 409.17it/s]


In [10]:
logit_attribution = LogitAttribution(
    dataset=dataset,
    model=logit_attr_model,
    batch_size=8,
    experiment="copyVSfact",
)

logit_attr_df = logit_attribution.run()
logit_attr_df

Attributing:   0%|          | 0/11 [00:00<?, ?it/s]

Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now


Attributing:   9%|▉         | 1/11 [00:04<00:48,  4.82s/it]

Tried to stack head results when they weren't cached. Computing head results now


Attributing:  18%|█▊        | 2/11 [00:05<00:22,  2.53s/it]

...

Tried to stack head results when they weren't cached. Computing head results now


Attributing:  91%|█████████ | 10/11 [06:38<00:49, 49.01s/it]

Tried to stack head results when they weren't cached. Computing head results now


Attributing: 100%|██████████| 11/11 [07:04<00:00, 38.62s/it]


,label,position,mem_mean,cp_mean,diff_mean,mem_std,cp_std,diff_std
0,L0H0,0,0.003548,0.000035,0.003513,0.025830,0.031719,0.041834
1,L0H0,1,0.008058,0.012324,-0.004266,0.022185,0.033281,0.037119
2,L0H0,2,0.012667,0.017614,-0.004946,0.018517,0.034533,0.036036
3,L0H0,3,0.018044,0.017581,0.000462,0.023233,0.028651,0.032691
4,L0H0,4,0.013997,0.010002,0.003995,0.013835,0.024615,0.024437
...,...,...,...,...,...,...,...,...
2422,all_heads,12,0.034782,0.046053,0.000000,0.126442,0.170084,0.000000
2423,all_heads_L10,12,0.092155,0.152299,0.000000,0.205348,0.327262,0.000000
2424,all_heads_L11,12,0.104581,0.127224,0.000000,0.176058,0.221034,0.000000
2425,all_heads_L7,12,0.043601,0.043012,0.000000,0.136977,0.143002,0.000000


In [11]:
def save_dataframe(folder_path, file_name, dataframe):
    print(folder_path)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    dataframe.to_csv(f"{folder_path}/{file_name}.csv", index=False)

In [13]:
save_dataframe(
    f"../results/{EXPERIMENT}/logit_attribution/{MODEL_FOLDER}",
    "logit_attribution_data",
    logit_attr_df,
)

../results/copyVSfactDomain/logit_attribution/gpt2_domain_Sensitive_Subjects


In [215]:
# !python /Users/udit/PycharmProjects/FACT/FACT_project/plotting_scripts/plot_logit_attribution_fig_3_4a.py \
#     {MODEL_NAME} {EXPERIMENT} {MODEL_FOLDER}